In [24]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 13.1 MB 9.1 MB/s 
     |████████████████████████████████| 90 kB 17.2 MB/s 
     |████████████████████████████████| 67 kB 9.4 MB/s 


In [8]:
%load_ext google.cloud.bigquery

In [27]:
%%bigquery coords
SELECT time_position, latitude, longitude

FROM `buoyant-site-244113.opensky.ingest`
WHERE TRIM(callsign) = 'DL118'

In [50]:
coords.head()

,time_position,latitude,longitude
0,2020-02-23 20:51:19+00:00,34.0636,-118.2046
1,2020-02-23 21:01:28+00:00,34.7757,-116.9742
2,2020-02-23 20:46:59+00:00,33.8495,-118.6041
3,2020-02-23 20:57:59+00:00,34.5153,-117.4162
4,2020-02-23 20:48:59+00:00,33.9295,-118.4396


In [65]:
%%bigquery coords
SELECT time_position, latitude, longitude

FROM `buoyant-site-244113.opensky.ingest`
WHERE RAND() < 0.001


In [22]:
%%bigquery
SELECT MAX(latitude) AS max_lat, MAX(longitude) AS max_long, MIN(latitude) AS min_lat, MIN(longitude) as min_long

FROM `buoyant-site-244113.opensky.ingest`
WHERE TRIM(callsign) = 'DL118'

,max_lat,max_long,min_lat,min_long
0,46.1063,-94.1968,33.8495,-118.6424


In [66]:
import numpy as np
import matplotlib.pyplot as plt

In [67]:
fig, ax = plt.subplots()

ax.scatter(coords['latitude'], coords['longitude'])

plt.show()

In [15]:
import altair as alt
from vega_datasets import data

In [60]:

airports = data.airports()
states = alt.topo_feature(data.us_10m.url, feature='states')

source = alt.topo_feature(data.world_110m.url, 'countries')

# US states background
base = alt.Chart(source).mark_geoshape(
    fill='#666666',
    stroke='white'
).properties(
    width=900,
    height=540
).project('equirectangular')

states = alt.topo_feature(data.us_10m.url, 'states')

background = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    title='US State Capitols',
    width=650,
    height=400
).project('albersUsa')

# airport positions on background
points = alt.Chart(coords).mark_circle(
    size=10,
    color='steelblue'
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    tooltip=['time_position']
)

base + points | background + points

alt.HConcatChart(...)